<a href="https://colab.research.google.com/github/uditbhaskar21/PySpark/blob/main/seatswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=73ec0ea06d03ae762fa1d43b1d876560da8a2031eb61bc84ca0e442925774cc6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
print(spark.version)


3.4.1


In [2]:
from pyspark.sql.functions import *

In [4]:
data = [('1','A'),('2','B'),('3','C'),('4','D'),('5','E')]
df = spark.createDataFrame(data,["id",'name'])
df.show()

+---+----+
| id|name|
+---+----+
|  1|   A|
|  2|   B|
|  3|   C|
|  4|   D|
|  5|   E|
+---+----+



In [5]:
from pyspark.sql.window import Window

In [9]:
df1 = df.withColumn('previous',lag('name').over(Window.orderBy('id')))
df2 = df1.withColumn('next',lead('name').over(Window.orderBy('id')))
df2.show()

+---+----+--------+----+
| id|name|previous|next|
+---+----+--------+----+
|  1|   A|    null|   B|
|  2|   B|       A|   C|
|  3|   C|       B|   D|
|  4|   D|       C|   E|
|  5|   E|       D|null|
+---+----+--------+----+



In [10]:
df3 = df2.withColumn('Exchanged',when(df2['id']%2 == 1, coalesce(df2['next'],df2['name'])). when(df2['id']%2 == 0, coalesce(df2['previous'],df2['name'])).otherwise(df2['name']))

In [11]:
df3.show()

+---+----+--------+----+---------+
| id|name|previous|next|Exchanged|
+---+----+--------+----+---------+
|  1|   A|    null|   B|        B|
|  2|   B|       A|   C|        A|
|  3|   C|       B|   D|        D|
|  4|   D|       C|   E|        C|
|  5|   E|       D|null|        E|
+---+----+--------+----+---------+



In [16]:
df4 = df3.drop('previous','next').withColumnRenamed('Exchanged','new')

In [17]:
df4.show()

+---+----+---+
| id|name|new|
+---+----+---+
|  1|   A|  B|
|  2|   B|  A|
|  3|   C|  D|
|  4|   D|  C|
|  5|   E|  E|
+---+----+---+



Spark SQL


In [ ]:
select id, name,
case
when id%2 == 1 then coalesce(lead(student) over (order by id), name)
WHEN id%2 == 0 then coalesce(lag(student) over (order by id), name)
else name end as exchanged
from table